# Example of alternative data access

Example of data access via the API

Load the libraries

In [ ]:
from osgeo import gdal
from gdalconst import GA_ReadOnly
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sg
import sys
import requests
import os
sys.path.insert(0,'/projects/Scripts')

# Increase figure size (can be modified for bigger or smaller figures):
plt.rcParams["figure.figsize"]=20,20

## 1. List the available datasets

GET ALL AVAILABLE datasetId and correlated subDasetId

In [ ]:
endpoint = "https://edav-das.val.esa-maap.org"
dat = requests.get(endpoint+"/opensearch/datasets")
if dat.status_code == 200: 
    response = []
    datasets = dat.json()["features"]
    for feature in datasets:
        elem = {}
        elem["datasetId"] = feature["datasetId"]
        elem["subDatasetIds"] = []
        for subdset in feature["subDataset"]:
            elem["subDatasetIds"].append(subdset)
        response.append(elem) 
        
    for ds in response:
        print(ds, "\n")

## 2. Discover the granule for a specific datasetId & subDatasetId

Below you see the code to discover the products/granules.

You can also filter using all the metadata querable prameters of the new metadata model implemented within the post_v1 (e.g. SceneType_value, gridType, ...) 

In [ ]:
datasetId = "AFRISAR_DLR"
subDatasetId = "AFRISAR_DLR_SLC_HH"
SceneType_value = "T2-0"
req = requests.get(endpoint + "/opensearch/search", params = {"SceneType_value":SceneType_value, "datasetId":datasetId, "subDatasetId":subDatasetId})
if req.status_code == 200:
    result = req.json()["features"]
    print(len(result))
    print(result)

## 3. Download the product/granule of interest

Below you see the code to download the products/granules of interest

In [ ]:
if len(result) > 0:
    for feature in result:
        print(feature)
        if "geolocated" in feature and feature["geolocated"] == 'True':
            wcs_query = endpoint + "/wcs?" + feature["sourceRasterGeo"]
        else:
            wcs_query = feature["source"]
            
        print(wcs_query)
        inputFilename = os.path.basename(wcs_query)
        r = requests.get(wcs_query, stream = True)
        if r.status_code == 200:
            with open(inputFilename, 'wb') as f:
                for chunk in r:
                    f.write(chunk)

Open the downloaded image for verification

In [ ]:
input_image_driver = gdal.Open(inputFilename, GA_ReadOnly)
input_image = input_image_driver.ReadAsArray()
input_image_driver = None
imgplot = plt.imshow(np.absolute(input_image))

## 4. To access the product/granule of interest directly in the bucket

Below you see the code to access the products/granules of interest

In [ ]:
import os
if len(result) > 0:
    for feature in result:
        if "geolocated" in feature and feature["geolocated"] == 'True':
            wcs_query = endpoint + "/wcs?" + feature["sourceRasterGeo"]
        else:
            wcs_query = feature["source"]
            
print(wcs_query)
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"

try:
    input_image_driver = gdal.Open("/vsicurl/" + wcs_query, GA_ReadOnly)
except Exception as e:
    print(e)
    
input_image = input_image_driver.ReadAsArray()
input_image_driver = None

Display SAR image in slant range geometry

In [ ]:
imgplot = plt.imshow(np.absolute(input_image))

# 5. Download a shapefile

In [ ]:
datasetId = "AFRISAR_DLR"
subDatasetId = "AFRISAR_DLR_ROI"
SceneType_value = "SAV1" 
req = requests.get(endpoint + "/opensearch/search", params = {"SceneType_value":SceneType_value, "datasetId":datasetId, "subDatasetId":subDatasetId})
if req.status_code == 200:
    result = req.json()["features"]
    print(len(result))
    print(result)

In [ ]:
import os
if len(result) > 0:
    for feature in result:
        for file in feature["source"].split(","):
            r = requests.get(file)
            if r.status_code == 200:
                with open(os.path.basename(file), 'wb') as f:
                    for chunk in r:
                        f.write(chunk)

# 6. Usefull command lines and tools

List all the allocated variables:

In [ ]:
who

List all the allocated variables and provide their type and content:

In [ ]:
whos